# (70 points) Problem of anomaly detection:
You are given the dataset of network user activity, and the task is to classify each user activity as normal or an attack. 
Attacks are also categorized as follows-
- Denial of Service (dos): Intruder tries to consume server resources as much as possible, so that normal users can’t get resources they need.
- Remote to Local (r2l): Intruder has no legitimate access to victim machine but tries to gain access.
- User to Root (u2r): Intruder has limited privilege access to victim machine but tries to get root privilege.
- Probe: Intruder tries to gain some information about victim machine.

Download dataset from here (http://researchweb.iiit.ac.in/~murtuza.bohra/intrusion_detection.zip).
Dataset contains 29 numerical features and five classes(one normal andfour attacks).

In [1]:
import pandas as pd
import numpy as np
import ipdb

In [2]:
df = pd.read_csv("data/intrusion_detection/data.csv")
df.head(5)
# df.describe()

,duration,service,src_bytes,dst_bytes,hot,num_failed_logins,num_compromised,num_root,num_file_creations,num_access_files,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,xAttack
0,0,25,193,441,0,0,0,0,0,0,...,255,1.00,0.00,0.07,0.04,0.00,0.04,0.0,0.0,normal
1,0,38,0,0,0,0,0,0,0,0,...,1,0.00,0.07,0.00,0.00,0.00,0.00,1.0,1.0,dos
2,0,25,167,9724,0,0,0,0,0,0,...,255,1.00,0.00,0.03,0.06,0.00,0.00,0.0,0.0,normal
3,0,20,1339,0,0,0,0,0,0,0,...,31,0.23,0.04,0.23,0.00,0.02,0.00,0.0,0.0,normal
4,0,37,0,0,0,0,0,0,0,0,...,25,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.0,dos


# Part-1: (20 points) PCA
Do dimensionality reduction using PCA on given dataset.
- Keep the tolerance of 10% (knee method), meaning reconstruction of the original data from the reduced dimensions in PCA space can be done with 10% error.
- You are only allowed to use eigen decomposition or SVD function from python library(do not use library function to compute PCA directly).

In [3]:
def choose_k(S, percent_variance_to_retain):
    """Chooses least no. of principal components(k) 
    which retains atleast the specified amount of variance in Xapprox."""
    k = 1
    
    while True:
        if np.sum(S[:k]) / np.sum(S) >= (percent_variance_to_retain/100): # :k where k is exclusive 
            return k
        k += 1

def PCA(X):
    """Performs PCA and returns data with reduced dimensions.
    Transform X to Z where dim(X) = mxn and dim(Z) = mxk"""
    mean = X.mean()
    std = X.std()

    # pre processing the data normalize and scale
    X = (X - mean) / std
    
    # compute covariance matrix
    # dim X = m x n
    m = len(X) # no of rows of data
    cov_matrix = (1/m) * np.dot(X.T, X) # nxm X mxn => nxn
    
    U, S, V = np.linalg.svd(cov_matrix)

    k = choose_k(S, 90)
    
    U_reduced = U[:, :k] # pick first k columns for new dimensions
    
    # dim(U_reduced) = nxk
    Z = np.dot(X, U_reduced)  # (m,n) * (n,k) => (m x k)
    return Z

In [4]:
X = df.drop(['xAttack'], axis=1)
print(X.shape)

Z = PCA(X)

print(Z.shape)

(24998, 29)
(24998, 14)


# Part-2: (15 points) k-means
- Use the reduced dimensions from the first part and perform K-means clustering with k equal to five(number of classes in the data).
- Also calculate the purity of clusters with given class label. Purity is the fraction of actual class samples in that cluster.
- You are not allowed to use inbuilt function for K-means.

In [5]:
# create a dataframe from dimensionally reduced data set
reduced_df=pd.DataFrame(Z)
reduced_df['xAttack'] = df['xAttack']  # FIXME: really slow sometimes
reduced_df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,xAttack
0,-2.509415,-0.952099,-0.080574,0.089473,-0.465558,0.264131,-0.073175,-0.122966,-0.115928,0.037054,-0.015485,0.085030,0.478752,0.380050,normal
1,1.193236,5.598092,-0.270790,1.130941,-0.813088,1.629181,0.541842,0.190282,0.243673,-0.079391,0.028093,0.064366,-0.430109,0.264010,dos
2,-2.447051,-0.908922,-0.060318,0.031070,-0.822722,0.170780,-0.098459,-0.161764,-0.088158,-0.084797,0.005994,0.050481,0.754346,0.429607,normal
3,-1.111736,-0.343088,0.004162,-0.266447,-0.166211,-0.596454,-0.015911,0.026994,0.161910,-0.061835,0.050150,-0.362922,0.542284,0.408395,normal
4,3.983887,-1.268837,-0.050827,0.252373,0.537136,0.561597,0.278500,0.038172,0.012687,-0.004357,-0.013466,0.041690,-0.072666,-0.071331,dos


In [6]:
# convert xAttack column to numerical values
reduced_df.xAttack, mapping_index = pd.Series(reduced_df.xAttack).factorize()
display(reduced_df.head())
print(mapping_index)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,xAttack
0,-2.509415,-0.952099,-0.080574,0.089473,-0.465558,0.264131,-0.073175,-0.122966,-0.115928,0.037054,-0.015485,0.085030,0.478752,0.380050,0
1,1.193236,5.598092,-0.270790,1.130941,-0.813088,1.629181,0.541842,0.190282,0.243673,-0.079391,0.028093,0.064366,-0.430109,0.264010,1
2,-2.447051,-0.908922,-0.060318,0.031070,-0.822722,0.170780,-0.098459,-0.161764,-0.088158,-0.084797,0.005994,0.050481,0.754346,0.429607,0
3,-1.111736,-0.343088,0.004162,-0.266447,-0.166211,-0.596454,-0.015911,0.026994,0.161910,-0.061835,0.050150,-0.362922,0.542284,0.408395,0
4,3.983887,-1.268837,-0.050827,0.252373,0.537136,0.561597,0.278500,0.038172,0.012687,-0.004357,-0.013466,0.041690,-0.072666,-0.071331,1


Index(['normal', 'dos', 'probe', 'r2l', 'u2r'], dtype='object')


In [7]:
class Cluster:
    """Data structure for a cluster."""
    def __init__(self, centroid):
        # centroid of this cluster
        self.centroid = centroid
        
        # points in this cluster
        self.points = np.array([])
        
        self.mean = 0
        
    def update_mean(self):
        self.mean = np.mean(self.points, axis=0)
        
    def add_point(self, point):
        if self.points.size > 0:
            self.points = np.vstack([self.points, point])            
        else:
            self.points = np.array(point)
            
    def clear(self):
        self.points = np.array([])

    def label(self):
        unique_labels, counts = np.unique(self.points[:,-1], return_counts=True)
        most_prominent_label = unique_labels[np.argmax(counts)]
        
        return most_prominent_label
        
    def purity(self):
        val, count = np.unique(self.points[:, -1], return_counts=True)
        return (100.0 * np.max(count)) / np.sum(count)


In [8]:
import random
        
class KMeans:
    def __init__(self, number_of_clusters=5):
        self.K = number_of_clusters
        self.clusters= []
    
    def fit(self, X, num_iters):
        X = X.values
        
        # randomly assign k cluster centroids
        # dim(centroids) = k x n
        centriod_indices = random.sample(population=range(len(X)), k=self.K)
        centriods = X[centriod_indices, :]
        
        # assign clusters with random chosen cluster centroids
        self.clusters = [Cluster(centroid) for centroid in centriods]
        
        m = len(X) # m = training set size
        
        for _ in range(num_iters):
            # reset all clusters by removing all points from each
            for cluster in self.clusters:
                cluster.clear()
            
            # find closest cluster for each of m points
            for row in X:
                # compute distance of this point from all clusters' centroids
                distances = np.array([self.euclidean_distance(row, cluster.centroid, dims=len(row)-1) for cluster in self.clusters])
                
                # add point to the closest cluster
                self.clusters[np.argmin(distances)].add_point(row)
            
            # reassign centroid to mean of points in that cluster
            for k in range(self.K):
                self.clusters[k].update_mean()
                
        
#         return self.centriods

    def print_cluster_details(self, mapping_index=None):
        for i, cluster in enumerate(self.clusters):
            label = cluster.label()
            print(f"i = {i}, size = {len(cluster.points)} purity = {cluster.purity()}, label={mapping_index[int(label)]}=={label}")


    def euclidean_distance(self, x, y, dims=None):
        """"""
        if dims is not None:
            x, y = x[:dims], y[:dims]

        return np.sqrt(np.sum((x - y) ** 2)) 


In [9]:
model = KMeans(number_of_clusters=5)
model.fit(reduced_df, num_iters=15)

In [10]:
model.print_cluster_details(mapping_index)

i = 0, size = 8233 purity = 99.1497631483056, label=normal==0.0
i = 1, size = 579 purity = 44.21416234887737, label=normal==0.0
i = 2, size = 900 purity = 77.55555555555556, label=probe==2.0
i = 3, size = 1636 purity = 75.0, label=normal==0.0
i = 4, size = 13650 purity = 64.00732600732601, label=dos==1.0
